In [1]:
import pickle

In [2]:
with open('train_qa.txt' , 'rb') as f:
    train_data = pickle.load(f)

In [3]:
with open('test_qa.txt' , 'rb') as f:
    test_data = pickle.load(f)

In [4]:
type(train_data)

list

In [5]:
len(train_data)

10000

In [6]:
type(test_data)

list

In [7]:
len(test_data)

1000

In [8]:
train_data[0]
#its essentially a tuple containing lists and strings in which the first list represents the story, the second list represents a query and the third string corresponds to the answer to that query

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [9]:
print(f"STORY: {' '.join(train_data[0][0])}\nQUERY: {' '.join(train_data[0][1])}\nANSWER: {train_data[0][2]}")

STORY: Mary moved to the bathroom . Sandra journeyed to the bedroom .
QUERY: Is Sandra in the hallway ?
ANSWER: no


In [10]:
#creating a vocabulary using sets by performing a union operation among all the stories and questions to extract the distinct words. We will also manually add tyhe words no and yes to the set since those are the only classes in the answer column.
all_data = train_data + test_data
vocab = set()
for story, query, answer in all_data:
    vocab = vocab.union(story)
    vocab = vocab.union(query)
vocab.add('no')
vocab.add('yes')

In [11]:
print(f"THE UNIQUE WORDS FROM THE DATASET THAT FORMS THE VOCABULARY ARE: \n\n{' | '.join(vocab)}")

THE UNIQUE WORDS FROM THE DATASET THAT FORMS THE VOCABULARY ARE: 

no | John | kitchen | grabbed | Is | down | the | bathroom | Mary | travelled | took | got | there | in | garden | went | moved | yes | left | ? | put | Sandra | Daniel | picked | journeyed | back | office | to | . | dropped | football | discarded | hallway | apple | bedroom | milk | up


In [12]:
len(vocab)

37

In [13]:
vocab_len = len(vocab) + 1

In [14]:
story_lengths = [len(data[0]) for data in all_data]

In [15]:
query_lengths = [len(data[1]) for data in all_data]

In [16]:
max_story_length = max(story_lengths)
max_story_length

156

In [17]:
max_query_length = max(query_lengths)
max_query_length

6

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
tokenizer = Tokenizer(filters=[])

In [21]:
tokenizer.fit_on_texts(vocab)

In [22]:
tokenizer.index_word

{1: 'no',
 2: 'john',
 3: 'kitchen',
 4: 'grabbed',
 5: 'is',
 6: 'down',
 7: 'the',
 8: 'bathroom',
 9: 'mary',
 10: 'travelled',
 11: 'took',
 12: 'got',
 13: 'there',
 14: 'in',
 15: 'garden',
 16: 'went',
 17: 'moved',
 18: 'yes',
 19: 'left',
 20: '?',
 21: 'put',
 22: 'sandra',
 23: 'daniel',
 24: 'picked',
 25: 'journeyed',
 26: 'back',
 27: 'office',
 28: 'to',
 29: '.',
 30: 'dropped',
 31: 'football',
 32: 'discarded',
 33: 'hallway',
 34: 'apple',
 35: 'bedroom',
 36: 'milk',
 37: 'up'}

In [23]:
tokenizer.word_index

{'no': 1,
 'john': 2,
 'kitchen': 3,
 'grabbed': 4,
 'is': 5,
 'down': 6,
 'the': 7,
 'bathroom': 8,
 'mary': 9,
 'travelled': 10,
 'took': 11,
 'got': 12,
 'there': 13,
 'in': 14,
 'garden': 15,
 'went': 16,
 'moved': 17,
 'yes': 18,
 'left': 19,
 '?': 20,
 'put': 21,
 'sandra': 22,
 'daniel': 23,
 'picked': 24,
 'journeyed': 25,
 'back': 26,
 'office': 27,
 'to': 28,
 '.': 29,
 'dropped': 30,
 'football': 31,
 'discarded': 32,
 'hallway': 33,
 'apple': 34,
 'bedroom': 35,
 'milk': 36,
 'up': 37}

In [24]:
import numpy as np


def vectorize(data, max_q_len = max_query_length, max_s_len = max_story_length, word_index = tokenizer.word_index):
    #List for vectorized stories
    vec_stories = []
    
    #List for vectorized queries
    vec_queries = []
    
    #List for vectorized answers
    vec_answers = []
    
    for st, qr, ans in data:
        vs = [word_index[word.lower()] for word in st]
        vq = [word_index[word.lower()] for word in qr]
        answ = np.zeros(len(word_index) + 1)
        answ[word_index[ans]] = 1
        vec_stories.append(vs)
        vec_queries.append(vq)
        vec_answers.append(answ)
    
    return (pad_sequences(vec_stories, max_s_len), pad_sequences(vec_queries, max_q_len), (np.array(vec_answers)))

In [25]:
vectorized_stories, vectorized_queries, vectorized_answers = vectorize(all_data)

In [26]:
vectorized_stories

array([[ 0,  0,  0, ...,  7, 35, 29],
       [ 0,  0,  0, ...,  7, 33, 29],
       [ 0,  0,  0, ...,  7,  8, 29],
       ...,
       [ 0,  0,  0, ...,  7, 34, 29],
       [ 0,  0,  0, ...,  7, 15, 29],
       [ 0,  0,  0, ..., 34, 13, 29]])

In [27]:
vectorized_queries

array([[ 5, 22, 14,  7, 33, 20],
       [ 5, 23, 14,  7,  8, 20],
       [ 5, 23, 14,  7, 27, 20],
       ...,
       [ 5,  9, 14,  7, 35, 20],
       [ 5, 22, 14,  7, 15, 20],
       [ 5,  9, 14,  7, 15, 20]])

In [28]:
vectorized_answers

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
sum(vectorized_answers)

array([   0., 5491.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
       5509.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.])

In [34]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Activation, Permute, Dropout, Embedding, add, dot, concatenate

In [35]:
input_seq = Input((max_story_length,))
question = Input((max_query_length,)) 

In [36]:
vocab_len

38

In [39]:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim = vocab_len, output_dim = 64))
input_encoder_m.add(Dropout(0.36))

In [40]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim = vocab_len, output_dim = max_query_length))
input_encoder_c.add(Dropout(0.36))

In [42]:
query_encoder_u = Sequential()
query_encoder_u.add(Embedding(input_dim = vocab_len, output_dim = 64, input_length = max_query_length))
query_encoder_u.add(Dropout(0.36))

In [43]:
input_encoded_m = input_encoder_m(input_seq)
input_encoded_c = input_encoder_c(input_seq)
query_encoded = query_encoder_u(question)

In [44]:
match = dot([input_encoded_m, query_encoded], axes = (2,2))
match = Activation('softmax')(match)

In [47]:
match

<KerasTensor: shape=(None, 156, 6) dtype=float32 (created by layer 'activation')>

In [45]:
response = add([match, input_encoded_c])
response = Permute((2,1))(response)

In [46]:
response

<KerasTensor: shape=(None, 6, 156) dtype=float32 (created by layer 'permute')>

In [52]:
answer = concatenate([response, query_encoded])

In [53]:
answer

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate')>

In [55]:
from tensorflow.keras.layers import LSTM
answer = LSTM(32)(answer)

In [57]:
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)
answer = Activation('softmax')(answer)

In [59]:
model = Model([input_seq, question], answer)

In [64]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [65]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 156)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 sequential_2 (Sequential)      (None, None, 64)     2432        ['input_1[0][0]']                
                                                                                                  
 sequential_5 (Sequential)      (None, 6, 64)        2432        ['input_2[0][0]']                
                                                                                            

In [66]:
input_train, queries_train, answers_train = vectorize(train_data)

In [67]:
input_test, queries_test, answers_test = vectorize(test_data)

In [77]:
history = model.fit([input_train, queries_train], answers_train, batch_size = 32, epochs = 100, validation_data = ([input_test, queries_test], answers_test))

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

Epoch 1/100
312/313 [============================>.] - ETA: 0s - loss: 0.6933 - accuracy: 0.5021

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

313/313 [==============================] - 5s 15ms/step - loss: 0.6933 - accuracy: 0.5016 - val_loss: 0.6933 - val_accuracy: 0.4950
Epoch 2/100
313/313 [==============================] - 4s 14ms/step - loss: 0.6932 - accuracy: 0.5084 - val_loss: 0.6933 - val_accuracy: 0.5030
Epoch 3/100
313/313 [==============================] - 4s 14ms/step - loss: 0.6935 - accuracy: 0.4951 - val_loss: 0.6935 - val_accuracy: 0.4930
Epoch 4/100
313/313 [==============================] - 4s 14ms/step - loss: 0.6931 - accuracy: 0.5093 - val_loss: 0.6936 - val_accuracy: 0.4930
Epoch 5/100
313/313 [==============================] - 4s 14ms/step - loss: 0.6931 - accuracy: 0.5067 - val_loss: 0.6938 - val_accuracy: 0.4950
Epoch 6/100
313/313 [==============================] - 4s 14ms/step - loss: 0.6935 - accuracy: 0.5048 - val_loss: 0.6940 - val_accuracy: 0.5060
Epoch 7/100
313/313 [==============================] - 5s 15ms/step - loss: 0.6932 - accuracy: 0.5076 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoc

In [81]:
res = model.predict([input_test, queries_test])

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

32/32 [==============================] - 1s 4ms/step


In [93]:
print(f"STORY: {' '.join(test_data[5][0])}\nQUERY: {' '.join(test_data[5][1])}\nTRUE ANSWER: {test_data[5][2]}")
print('\n')

max_val = np.argmax(res[5])
for key, val in tokenizer.word_index.items():
    if val == max_val:
        k = key
        break
print(f"PREDICTED ANSWER: {k}\nCERTAINITY: {np.round(res[0][max_val],3)*100}%")

STORY: Daniel went back to the kitchen . Mary grabbed the apple there .
QUERY: Is Daniel in the office ?
TRUE ANSWER: no


PREDICTED ANSWER: no
CERTAINITY: 99.6999979019165%
